The following code will use the Nominatim (Names) API to append a latitude and a longitude column to both the Tim Ryan & J.D. Vance data

In [24]:
import pandas as pd
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geoapiUser")

In [214]:
ryan = pd.read_csv('data/tim_ryan.csv')
vance = pd.read_csv('data/jd_vance.csv')

In [212]:
# API References
endpoint = 'https://nominatim.openstreetmap.org/search?'
footer = '&format=json&limit=1&email=jasonbconley@outlook.com'

# ACT BLUE Lookup
query = 'city={}&state={}'.format('Cambridge','MA')
search_url = endpoint + query + footer
response = requests.get(search_url)
details = json5.loads(response.text)

act_blue_lat = zip_dict[0]['lat']
act_blue_lon = zip_dict[0]['lon']

# Data Lookups
coord_dict = dict()

lat = []
lon = []

# Action
for index, row in ryan.iterrows():
    city = row['City']
    state = row['State']
    
    # Some entries are poorly structured for credit statements, the length check ignores them
    if (len(state) == 2):
        if (city == 'Cambridge' and state == 'MA'):
            lat.append(act_blue_lat)
            lon.append(act_blue_lon)

        elif ( ( city + ' ' + state ) in coord_dict.keys()): 
            latVal, lonVal = coord_dict[city + ' ' + state].split(' ')
            lat.append(latVal)
            lon.append(lonVal)

        else:
            query = 'city={}&state={}'.format(city,state)
            search_url = endpoint + query + footer
            response = requests.get(search_url)

            details = json5.loads(response.text)

            if (len(details) > 0):
                latVal = details[0]['lat']
                lonVal = details[0]['lon']
                lat.append(latVal)
                lon.append(lonVal)
                coord_dict[city + ' ' + state] = ( latVal + ' ' + lonVal )

ryan['lat'] = lat
ryan['lon'] = lon

ValueError: Length of values (79770) does not match length of index (80043)

In [213]:
coord_dict = dict()

lat = []
lon = []

for index, row in vance.iterrows():
    city = row['City']
    state = row['State']
    
    if (len(state) == 2):
        if ( ( city + ' ' + state ) in coord_dict.keys()): 
            latVal, lonVal = coord_dict[city + ' ' + state].split(' ')
            lat.append(latVal)
            lon.append(lonVal)

        else:
            query = 'city={}&state={}'.format(city,state)
            search_url = endpoint + query + footer
            response = requests.get(search_url)

            details = json5.loads(response.text)

            if (len(details) > 0):
                latVal = details[0]['lat']
                lonVal = details[0]['lon']
                lat.append(latVal)
                lon.append(lonVal)
                coord_dict[city + ' ' + state] = ( latVal + ' ' + lonVal )

vance['lat'] = lat
vance['lon'] = lon

KeyboardInterrupt: 

In [ ]:
ryan.dropna(axis = 0, subset = ['lat', 'lon'], inplace = True)
vance.dropna(axis = 0, subset = ['lat', 'lon'], inplace = True)

flag = []
for index, row in ryan.iterrows():
    if (row['state'] == 'OH'):
        flag.append('In state')
    else:
        flag.append('Out of state')

ryan['State flag'] = flag

flag = []
for index, row in vance.iterrows():
    if (row['state'] == 'OH'):
        flag.append('In state')
    else:
        flag.append('Out of state')

vance['State flag'] = flag